In [ ]:
"""
Go SHORT on MNQ (Sell 1 contract) with SL and TP
- 40 point Take Profit
- 5 point Stop Loss
Uses bracket orders for automatic SL/TP
"""
import os
import requests
from dotenv import load_dotenv

load_dotenv()

USERNAME = os.getenv("TOPSTEP_USERNAME")
KEY = os.getenv("TOPSTEP_KEY")
API_BASE = "https://api.topstepx.com/api"

# Configuration
TP_POINTS = 40  # Take Profit in points
SL_POINTS = 5   # Stop Loss in points

print("=" * 60)
print("GO SHORT (SELL) - MNQ with SL/TP")
print(f"Take Profit: {TP_POINTS} points | Stop Loss: {SL_POINTS} points")
print("=" * 60)

# Authenticate
print("\n[1/3] Authenticating...")
auth_response = requests.post(
    f"{API_BASE}/Auth/loginKey",
    json={"userName": USERNAME, "apiKey": KEY},
    headers={"accept": "text/plain", "Content-Type": "application/json"},
    timeout=15
)

if auth_response.status_code != 200:
    print(f"❌ Auth failed")
    exit(1)

token = auth_response.json()["token"]
print(f"✅ Authenticated")

headers = {
    "Authorization": f"Bearer {token}",
    "accept": "text/plain",
    "Content-Type": "application/json"
}

# Get Account
print("\n[2/3] Getting account...")
search_response = requests.post(
    f"{API_BASE}/Account/search",
    json={"onlyActiveAccounts": True},
    headers=headers,
    timeout=15
)

account = search_response.json()["accounts"][0]
account_id = account["id"]
print(f"✅ Account: {account['name']}")
print(f"   Balance: ${account.get('balance', 0):,.2f}")

# Place SELL order with bracket orders
print("\n[3/3] Placing SELL order with SL/TP brackets...")
print("Side: 1 (SELL)")

order_payload = {
    "accountId": account_id,
    "contractId": "CON.F.US.MNQ.H26",
    "type": 2,    # Market
    "side": 1,    # SELL
    "size": 1,
    "stopLossBracket": {
        "ticks": SL_POINTS * 4,  # Positive: 5 points = 20 ticks (stop above entry)
        "type": 4  # Stop order
    },
    "takeProfitBracket": {
        "ticks": -(TP_POINTS * 4),  # Negative: -40 points = -160 ticks (profit below entry)
        "type": 1  # Limit order
    }
}

print(f"Payload: {order_payload}")

response = requests.post(
    f"{API_BASE}/Order/place",
    json=order_payload,
    headers=headers,
    timeout=15
)

print(f"\nStatus Code: {response.status_code}")
result = response.json()
print(f"Response: {result}")

if result.get('success'):
    print(f"\n✅ SUCCESS! Order ID: {result.get('orderId')}")
    print(f"\nPosition: SHORT 1 MNQ")
    print(f"Take Profit: {TP_POINTS} points below entry (Limit order)")
    print(f"Stop Loss: {SL_POINTS} points above entry (Stop order)")
else:
    print(f"\n❌ FAILED: {result.get('errorMessage')}")

print("\n" + "=" * 60)